`Дисциплина: Методы и технологии машинного обучения`   
`Уровень подготовки: бакалавриат`   
`Направление подготовки: 01.03.02 Прикладная математика и информатика`   
`Семестр: осень 2021/2022`   




# Лабораторная работа №3: Линейные модели. Кросс-валидация. 

В практических примерах ниже показано:   

* как пользоваться инструментами предварительного анализа для поиска линейных взаимосвязей 
* как строить и интерпретировать линейные модели с логарифмами  
* как оценивать точность моделей с перекрёстной проверкой (LOOCV, проверка по блокам)

*Модели*: множественная линейная регрессия 
*Данные*: `insurance` (источник: <https://www.kaggle.com/mirichoi0218/insurance/version/1>)

In [ ]:
# настройка ширины страницы блокнота .......................................
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Указания к выполнению


## Загружаем пакеты

In [ ]:
# загрузка пакетов: инструменты --------------------------------------------
#  работа с массивами
import numpy as np
#  фреймы данных
import pandas as pd
#  графики
import matplotlib as mpl
#  стили и шаблоны графиков на основе matplotlib
import seaborn as sns
# перекодировка категориальных переменных
from sklearn.preprocessing import LabelEncoder
#  тест Шапиро-Уилка на нормальность распределения
from scipy.stats import shapiro
#  для таймера
import time

# загрузка пакетов: модели -------------------------------------------------
#  линейные модели
import sklearn.linear_model as skl_lm
#  расчёт MSE
from sklearn.metrics import mean_squared_error
#  кросс-валидация
from sklearn.model_selection import train_test_split, LeaveOneOut 
from sklearn.model_selection import KFold, cross_val_score
#  полиномиальные модели
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# константы
#  ядро для генератора случайных чисел
my_seed = 9212
#  создаём псевдоним для короткого обращения к графикам
plt = mpl.pyplot
# настройка стиля и отображения графиков
#  примеры стилей и шаблонов графиков: 
#  http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html
mpl.style.use('seaborn-whitegrid')
sns.set_palette("Set2")
# раскомментируйте следующую строку, чтобы посмотреть палитру
# sns.color_palette("Set2")

## Загружаем данные

Набор данных `insurance` в формате .csv доступен для загрузки по адресу: <https://raw.githubusercontent.com/aksyuk/MTML/main/Labs/data/insurance.csv>. Справочник к данным: <https://github.com/aksyuk/MTML/blob/main/Labs/data/CodeBook_insurance.md>.  
Загружаем данные во фрейм и кодируем категориальные переменные.  

In [ ]:
# читаем таблицу из файла .csv во фрейм
fileURL = 'https://raw.githubusercontent.com/aksyuk/MTML/main/Labs/data/insurance.csv'
DF_raw = 

# выясняем размерность фрейма
print('Число строк и столбцов в наборе данных:\n', DF_raw.shape)

In [ ]:
# первые 5 строк фрейма


In [ ]:
# типы столбцов фрейма


Проверим, нет ли в таблице пропусков.  

In [ ]:
# считаем пропуски в каждом столбце


Пропусков не обнаружено.  

In [ ]:
# кодируем категориальные переменные
#  пол
sex_dict = 
DF_raw['sexFemale'] = 

#  курильщик
yn_dict = 
DF_raw['smokerYes'] = 

# находим уникальные регионы


In [ ]:
#  добавляем фиктивные на регион: число фиктивных = число уникальных - 1
df_dummy = 
df_dummy.head(5)

In [ ]:
# объединяем с исходным фреймом
DF_all = 

# сколько теперь столбцов
DF_all.shape

In [ ]:
# смотрим первые 8 столбцов
DF_all.iloc[:, :8].head(5)

In [ ]:
# смотрим последние 5 столбцов
DF_all.iloc[:, 8:].head(5)

In [ ]:
# оставляем в наборе данных только то, что нужно 
#  (плюс метки регионов для графиков)
DF_all = DF_all[['charges', 'age', 'sexFemale', 'bmi', 'children', 'smokerYes', 
                 'region_northwest', 'region_southeast',
                 'region_southwest', 'region']]

# перекодируем регион в числовой фактор, 
#  чтобы использовать на графиках
class_le = 
DF_all['region'] = 

DF_all.columns

In [ ]:
DF_all.dtypes

In [ ]:
# удаляем фрейм-исходник


Прежде чем переходить к анализу данных, разделим фрейм на две части: одна (90%) станет основой для обучения моделей, на вторую (10%) мы сделаем прогноз по лучшей модели.  

In [ ]:
# данные для построения моделей
DF = 

# данные для прогнозов
DF_predict = 

## Предварительный анализ данных   

### Считаем описательные статистики   

Рассчитаем описательные статистики для непрерывных переменных. Из таблицы ниже можно видеть, что переменная `charges`, которая является зависимой переменной модели, сильно отличается по масштабу от всех остальных.    Также заметим, что из всех объясняющих только переменная `children` принимает нулевые значения. Остальные показатели положительны.  

In [ ]:
# описательные статистики для непрерывных переменных


### Строим графики  

Посмотрим на графики взаимного разброса непрерывных переменных. 

In [ ]:
# матричный график разброса с линиями регрессии

plt.show()

Судя по этим графикам:  
* распределение зависимой `charges` не является нормальным;  
* из всех объясняющих нормально распределена только `bmi`;  
* имеется три уровня стоимости страховки, что заметно на графиках разброса `charges` от `age`;  
* облако наблюдений на графике `charges` от `bmi` делится на две неравные части;  
* объясняющая `children` дискретна, что очевидно из её смысла: количество детей;  
* разброс значений `charges` у застрахованных с количеством детей 5 (максимум из таблицы выше) намного меньше, чем у остальных застрахованных.  

Наблюдаемые закономерности могут объясняться влиянием одной или нескольких из фиктивных объясняющих переменных. Построим график, раскрасив точки цветом в зависимости от пола застрахованного лица.  

In [ ]:
# матричный график разброса с цветом по полу


plt.show()

Теперь покажем цветом на графиках отношение застрахованых лиц к курению.

In [ ]:
# матричный график разброса с цветом по smokerYes


plt.show()

Покажем с помощью цвета на графиках регионы.

In [ ]:
# матричный график разброса с цветом по region


plt.show()

Нарисуем график отдельно по `region_southeast`.  

In [ ]:
# матричный график разброса с цветом по региону southeast


plt.show()

Посмотрим на корреляционные матрицы непрерывных переменных фрейма. 

In [ ]:
# корреляционная матрица по всем наблюдениям
corr_mat = 
corr_mat.style.background_gradient(cmap='coolwarm').set_precision(2)

Посчитаем корреляционные матрицы для курящих и некурящих застрахованных лиц.  

In [ ]:
# корреляционная матрица по классу курильщиков
corr_mat =

corr_mat.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
# корреляционная матрица по классу не курильщиков
corr_mat = 

corr_mat.style.background_gradient(cmap='coolwarm').set_precision(2)



### Логарифмируем зависимую переменную  

Важным допущением линейной регрессии является нормальность зависимой переменной. Чтобы добиться нормального распределения, используют логарифмирование либо преобразование Бокса-Кокса. В этой лабораторной остановимся на логарифмировании.   

In [ ]:
# логарифмируем зависимую переменную
DF['log_charges'] = 

# описательные статистики для непрерывных показателей
DF[['charges', 'log_charges', 'age', 'bmi', 'children']].describe()

Проведём формальные тесты на нормальность.  

In [ ]:
# тестируем на нормальность
for col in ['charges', 'log_charges']:
    stat, p = shapiro(DF[col])
    print(col, 'Statistics=%.2f, p=%.4f' % (stat, p))
    # интерпретация
    alpha = 0.05
    if p > alpha:
        print('Распределение нормально (H0 не отклоняется)\n')
    else:
        print('Распределение не нормально (H0 отклоняется)\n')

Логарифмирование меняет взаимосвязи между переменными.   

In [ ]:
# матричный график разброса с цветом по smokerYes
sns.pairplot(DF[['log_charges', 'age', 'bmi', 'children', 
                 'smokerYes']], hue='smokerYes')
plt.show()

In [ ]:
# корреляционная матрица по классу не курильщиков
corr_mat = DF.loc[DF['smokerYes'] == 0][['log_charges', 'age', 
                                         'bmi', 'children']].corr()
corr_mat.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
# корреляционная матрица по классу курильщиков
corr_mat = DF.loc[DF['smokerYes'] == 1][['log_charges', 'age', 
                                         'bmi', 'children']].corr()
corr_mat.style.background_gradient(cmap='coolwarm').set_precision(2)

## Строим модели регрессии

### Спецификация моделей  
По итогам предварительного анализа данных можно предложить следующие спецификации линейных регрессионных моделей:  

1. `fit_lm_1`: $\hat{charges} = \hat{\beta_0} + \hat{\beta_1} \cdot smokerYes + \hat{\beta_2} \cdot age + \hat{\beta_3} \cdot bmi$
1. `fit_lm_2`: $\hat{charges} = \hat{\beta_0} + \hat{\beta_1} \cdot smokerYes + \hat{\beta_2} \cdot age \cdot smokerYes + \hat{\beta_3} \cdot bmi$
1. `fit_lm_3`: $\hat{charges} = \hat{\beta_0} + \hat{\beta_1} \cdot smokerYes + \hat{\beta_2} \cdot bmi \cdot smokerYes + \hat{\beta_3} \cdot age$
1. `fit_lm_4`: $\hat{charges} = \hat{\beta_0} + \hat{\beta_1} \cdot smokerYes + \hat{\beta_2} \cdot bmi \cdot smokerYes + \hat{\beta_3} \cdot age \cdot smokerYes$

1. `fit_lm_1_log`: то же, что `fit_lm_1`, но для зависимой $\hat{log\_charges}$  
1. `fit_lm_2_log`: то же, что `fit_lm_2`, но для зависимой $\hat{log\_charges}$
1. `fit_lm_3_log`: то же, что `fit_lm_3`, но для зависимой $\hat{log\_charges}$
1. `fit_lm_4_log`: то же, что `fit_lm_4`, но для зависимой $\hat{log\_charges}$

Кроме того, добавим в сравнение модели зависимости `charges` и `log_sharges` от всех объясняющих переменных: `fit_lm_0` и `fit_lm_0_log` соответственно.  


### Обучение и интерпретация  

Создаём матрицы значений объясняющих переменных ( $X$ ) и вектора значений зависимой ( $y$ ) для всех моделей.  

In [ ]:
# данные для моделей 1, 5
df1 = DF[['charges', 'smokerYes', 'age', 'bmi']]

# данные для моделей 2, 6
df2 = DF[['charges', 'smokerYes', 'age', 'bmi']]
df2.loc[:, 'age_smokerYes'] = 
df2 = 

# данные для моделей 3, 7
df3 = DF[['charges', 'smokerYes', 'age', 'bmi']]
df3.loc[:, 'bmi_smokerYes'] = 
df3 = 

# данные для моделей 4, 8
df4 = DF[['charges', 'smokerYes', 'age', 'bmi']]
df4.loc[:, 'age_smokerYes'] = 
df4.loc[:, 'bmi_smokerYes'] = 
df4 = 

# данные для моделей 9, 10
df0 = 

Построим модели от всех объясняющих переменных на всех наблюдениях `DF`, чтобы проинтерпретировать параметры. В модели для зависимой переменной `charges` интерпретация стандартная:  

1. Константа – базовый уровень зависимой переменной, когда все объясняющие равны 0.  
2. Коэффициент при объясняющей переменной $X$ показывает, на сколько своих единиц измерения изменится $Y$, если $X$ увеличится на одну свою единицу измерения.  

In [ ]:
lm = skl_lm.LinearRegression()

# модель со всеми объясняющими, y
X = 
y = 
fit_lm_0 = 
print('модель fit_lm_0:\n', 
      'константа ', np.around(fit_lm_0.intercept_, 3),
      '\n объясняющие ', list(X.columns.values),
      '\n коэффициенты ', np.around(fit_lm_0.coef_, 3))

In [ ]:
# оценим MSE на обучающей
#  прогнозы
y_pred = 
MSE = 
MSE

С интрпретацией модели на логарифме $Y$ дела обстоят сложнее:  
1. Константу сначала надо экспоненциировать, далее интерпретировать как для обычной модели регрессии.  
1. Коэффициент при $X$ нужно экспоненциировать, затем вычесть из получившегося числа 1, затем умножить на 100. Результат показывает, на сколько процентов изменится (увеличится, если коэффициент положительный, и уменьшится, если отрицательный) зависимая переменная, если $X$ увеличится на одну свою единицу измерения.  

In [ ]:
# модель со всеми объясняющими, y_log
X = df0.drop(['charges'], axis=1)
y = np.log(df0.charges).values.reshape(-1, 1)
fit_lm_0_log = lm.fit(X, y)
print('модель fit_lm_0_log:\n', 
      'константа ', np.around(fit_lm_0_log.intercept_, 3),
      '\n объясняющие ', list(X.columns.values),
      '\n коэффициенты ', np.around(fit_lm_0_log.coef_, 3))

In [ ]:
# пересчёт коэффициентов для их интерпретации


In [ ]:
# оценим MSE на обучающей
#  прогнозы
y_pred = fit_lm_0_log.predict(X)
MSE_log = sum((np.exp(y) - np.exp(y_pred).reshape(-1, 1))**2) / len(y)
MSE_log

In [ ]:
print('MSE_train модели для charges меньше MSE_train',
     'модели для log(charges) в ', np.around(MSE_log / MSE, 1), 'раз')

### Оценка точности

#### LOOCV  

Сделаем перекрёстную проверку точности моделей по одному наблюдению.  

In [ ]:
# LeaveOneOut CV
loo = 

# модели для y
scores = list()
# таймер
tic = 
for df in [df0, df1, df2, df3, df4] :
    
    X = 
    y = 
    score = 
    
    scores.append(score)

# таймер
toc = 
print(f"Расчёты методом LOOCV заняли {} секунд")

In [ ]:
# модели для y_log
scores_log = list()
# таймер
tic = time.perf_counter()
for df in [df0, df1, df2, df3, df4] :
    loo.get_n_splits(df)
    X = df.drop(['charges'], axis=1)
    y = np.log(df.charges)
    score = cross_val_score(lm, X, y, cv=loo, n_jobs=1,
                            scoring='neg_mean_squared_error').mean()
    scores_log.append(score)

# таймер
toc = time.perf_counter()
print(f"Расчёты методом LOOCV заняли {toc - tic:0.2f} секунд")

Сравним ошибки для моделей на исходных значениях `charges` с ошибками моделей на логарифме.  

In [ ]:
[np.around(-x, 2) for x in scores]

In [ ]:
[np.around(-x, 3) for x in scores_log]

Определим самые точные модели отдельно на `charges` и на `log_charges`.  

In [ ]:
# самая точная на charges
fits = ['fit_lm_0', 'fit_lm_1', 'fit_lm_2', 'fit_lm_3', 'fit_lm_4']
print('Наименьшая ошибка на тестовой с LOOCV у модели',
      fits[scores.index(max(scores))], 
      ':\nMSE_loocv =', np.around(-max(scores), 0))

In [ ]:
# самая точная на log(charges)
fits = ['fit_lm_0_log', 'fit_lm_1_log', 'fit_lm_2_log', 
        'fit_lm_3_log', 'fit_lm_4_log']
print('Наименьшая ошибка на тестовой с LOOCV у модели',
      fits[scores_log.index(max(scores_log))], 
      ':\nMSE_loocv =', np.around(-max(scores_log), 3))

#### Перекрёстная проверка по блокам    

Теоретически этот метод менее затратен, чем LOOCV. Проверим на наших моделях.   

In [ ]:
# Перекрёстная проверка по 10 блокам
folds = 

# ядра для разбиений перекрёстной проверкой
r_state = 

# модели для y
scores = list()
# таймер
tic = time.perf_counter()
i = 0
for df in [df0, df1, df2, df3, df4] :
    X = df.drop(['charges'], axis=1)
    y = df.charges
    kf_10 = 
    
    score = cross_val_score(lm, X, y, cv=kf_10,
                            scoring='neg_mean_squared_error').mean()
    scores.append(score)
    i+=1

# таймер
toc = time.perf_counter()
print(f"Расчёты методом CV по 10 блокам заняли {toc - tic:0.2f} секунд")

In [ ]:
# Перекрёстная проверка по 10 блокам
folds = 10

# ядра для разбиений перекрёстной проверкой
r_state = np.arange(my_seed, my_seed + 9)

# модели для y
scores_log = list()
# таймер
tic = time.perf_counter()
i = 0
for df in [df0, df1, df2, df3, df4] :
    X = df.drop(['charges'], axis=1)
    y = np.log(df.charges)
    kf_10 = KFold(n_splits=folds, random_state=r_state[i],
                 shuffle=True)
    score = cross_val_score(lm, X, y, cv=kf_10,
                            scoring='neg_mean_squared_error').mean()
    scores_log.append(score)
    i+=1

# таймер
toc = time.perf_counter()
print(f"Расчёты методом CV по 10 блокам заняли {toc - tic:0.2f} секунд")

In [ ]:
# самая точная на charges
fits = ['fit_lm_0', 'fit_lm_1', 'fit_lm_2', 'fit_lm_3', 'fit_lm_4']
print('Наименьшая ошибка на тестовой с k-fold10 у модели',
      fits[scores.index(max(scores))], 
      ':\nMSE_kf10 =', np.around(-max(scores), 0))

In [ ]:
# самая точная на log(charges)
fits = ['fit_lm_0_log', 'fit_lm_1_log', 'fit_lm_2_log', 
        'fit_lm_3_log', 'fit_lm_4_log']
print('Наименьшая ошибка на тестовой с k-fold10 у модели',
      fits[scores_log.index(max(scores_log))], 
      ':\nMSE_kf10 =', np.around(-max(scores_log), 3))

Можно убедиться, что оценка MSE методом перекрёстной проверки по 10 блокам даёт результаты, практически идентичные методу LOOCV. При этом скорость у первого метода при 1204 наблюдениях выше на два порядка.

Самой точной среди моделей для `charges` оказалась `fit_lm_3`, а среди моделей для `charges_log` – `fit_lm_0_log`. Оценим точность прогноза по этим моделям на отложенные наблюдения.   

In [ ]:
# прогноз по fit_lm_3
#  модель на всех обучающих наблюдениях
X = df3.drop(['charges'], axis=1)
y = df3.charges.values.reshape(-1, 1)
fit_lm_3  = 

#  значения y на отложенных наблюдениях
y = DF_predict[['charges']].values.reshape(-1, 1)
#  матрица объясняющих на отложенных наблюдениях
X = DF_predict[['smokerYes', 'age', 'bmi']]
X.loc[:, 'bmi_smokerYes'] = X.loc[:, 'bmi'] * X.loc[:, 'smokerYes']
X = X.drop(['bmi'], axis=1)
#  прогнозы
y_pred = 

# ошибка
MSE = 
print('MSE модели fit_lm_3 на отложенных наблюдениях = %.2f' % MSE)

In [ ]:
# прогноз по fit_lm_log_0
# модель
X = df0.drop(['charges'], axis=1)
y = np.log(df0.charges).values.reshape(-1, 1)
fit_lm_0_log = 

#  значения y на отложенных наблюдениях
y = np.log(DF_predict[['charges']].values.reshape(-1, 1))
#  матрица объясняющих на отложенных наблюдениях
X = DF_predict.drop(['charges', 'region'], axis=1)

#  прогнозы
y_pred = 

# ошибка
MSE_log = 
print('MSE модели fit_lm_0_log на отложенных наблюдениях = %.2f' % MSE_log)

Очевидно, на выборке для прогноза точнее модель `fit_lm_3`:  
$\hat{charges} = \hat{\beta_0} + \hat{\beta_1} \cdot smokerYes + \hat{\beta_2} \cdot bmi \cdot smokerYes + \hat{\beta_3} \cdot age$

In [ ]:
print('модель fit_lm_3:\n', 
      'константа ', np.around(fit_lm_3.intercept_, 3),
      '\n объясняющие ', list(df3.drop(['charges'], axis=1).columns.values),
      '\n коэффициенты ', np.around(fit_lm_3.coef_, 3))

# Источники 

1. *James G., Witten D., Hastie T. and Tibshirani R.*  An Introduction to Statistical Learning with Applications in R. URL: [http://www-bcf.usc.edu/~gareth/ISL/ISLR%20First%20Printing.pdf](https://drive.google.com/file/d/15PdWDMf9hkfP8mrCzql_cNiX2eckLDRw/view?usp=sharing)     
1. Рашка С. Python и машинное обучение: крайне необходимое пособие по новейшей предсказательной аналитике, обязательное для более глубокого понимания методологии машинного обучения / пер. с англ. А.В. Логунова. – М.: ДМК Пресс, 2017. – 418 с.: ил.
1. Interpreting Log Transformations in a Linear Model / virginia.edu. URL: <https://data.library.virginia.edu/interpreting-log-transformations-in-a-linear-model/>  
1. Python Timer Functions: Three Ways to Monitor Your Code / realpython.com. URL: <https://realpython.com/python-timer/>  